In [1]:
import mlflow
import pickle
import pandas as pd 
import numpy as np
import datetime
import ssl

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder


ssl._create_default_https_context = ssl._create_unverified_context

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("GenAI-Course-Simple-ANN-Classifier")

<Experiment: artifact_location='mlflow-artifacts:/336063370036033104', creation_time=1743665023308, experiment_id='336063370036033104', last_update_time=1743665023308, lifecycle_stage='active', name='GenAI-Course-Simple-ANN-Classifier', tags={}>

In [2]:
df = pd.read_csv('/Users/sarthakagarwal/Dropbox/study material/python/genAI/Udemy GenAI/simple RNN/data/Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [6]:
## dropping irrelevant feartures
df = df.drop(
    [
        "RowNumber",
        "CustomerId",
        "Surname"
    ],
    axis = 1
)


In [7]:
# Encoding categorical data
label_encoder = LabelEncoder()
df["Gender"] = label_encoder.fit_transform(df["Gender"])

# One hot encode Geography colunm
one_hot_encoder = OneHotEncoder()

geo_encoder = one_hot_encoder.fit_transform(df[["Geography"]])
geo_encoder_df = pd.DataFrame(
    geo_encoder.toarray(),
    columns = one_hot_encoder.get_feature_names_out(["Geography"])
    )
df = pd.concat([df, geo_encoder_df], axis = 1)
df.drop(["Geography"], axis = 1, inplace = True)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
## Save the encoder and pickle

with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder, file)
  
with open("one_hot_encoder_geography.pkl", "wb") as file:
    pickle.dump(one_hot_encoder, file)

In [9]:


# Divide data in independent and dependent variables 

x = df.drop(
    [
        "Exited"
    ],
    axis=1
)
y = df["Exited"].values.ravel()


# Making train test split

x_train, x_test, y_train, y_test = train_test_split(
    x,
    y,
    random_state=42,
    shuffle=True,
    test_size=0.2,
    train_size=0.8
)

#Scaling Data
scalar = StandardScaler()
x_train = scalar.fit_transform(x_train)
x_test = scalar.transform(x_test)

# saving scaled data
with open("scalar.pkl", "wb") as file:
    pickle.dump(scalar, file)
    



In [10]:
x_train.shape

(8000, 12)

In [11]:
model = Sequential(
    [
        Dense(128, activation="relu", input_shape=(x_train.shape[1],)),
        Dense(64, activation="relu"),
        Dense(32, activation="relu"),
        Dense(16, activation="relu"),
        Dense(8, activation="relu"),
        Dense(4, activation="relu"),
        Dense(2, activation="relu"),
        Dense(1, activation="relu"),
    ]
)

model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=BinaryCrossentropy(),
    metrics = ["accuracy"]
)

2025-04-03 13:25:30.514266: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-03 13:25:30.514410: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-03 13:25:30.514423: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-03 13:25:30.514668: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-03 13:25:30.514974: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1664      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 4)                 36        
                                                                 
 dense_6 (Dense)             (None, 2)                 1

In [13]:

log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [14]:
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [15]:
mlflow.autolog()
with mlflow.start_run():
    history = model.fit(
        x_train,
        y_train,
        epochs=100,
        validation_data=(x_test, y_test),
        callbacks=[
            tensorboard_callback,
            early_stopping
            ],
     )
    

2025/04/03 13:25:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/04/03 13:25:31 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/04/03 13:25:31 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.


Epoch 1/100


2025-04-03 13:25:32.045409: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-03 13:25:32.110522: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


250/250 [==============================] - 3s 7ms/step - loss: 6.7186 - accuracy: 0.5545 - val_loss: 6.4459 - val_accuracy: 0.5775
Epoch 2/100
 18/250 [=>............................] - ETA: 1s - loss: 6.0247 - accuracy: 0.6042

/Users/sarthakagarwal/Dropbox/study material/python/genAI/Udemy GenAI/genai/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


250/250 [==============================] - 2s 7ms/step - loss: 6.6479 - accuracy: 0.5646 - val_loss: 6.3603 - val_accuracy: 0.5840
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 6.2194 - accuracy: 0.5932 - val_loss: 6.3603 - val_accuracy: 0.5840
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 6.2194 - accuracy: 0.5932 - val_loss: 6.3603 - val_accuracy: 0.5840
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 6.2194 - accuracy: 0.5932 - val_loss: 6.3603 - val_accuracy: 0.5840
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 6.2194 - accuracy: 0.5932 - val_loss: 6.3603 - val_accuracy: 0.5840
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 6.2194 - accuracy: 0.5932 - val_loss: 6.3603 - val_accuracy: 0.5840
Epoch 8/100
250/250 [==============================] - 1s 5ms/step - loss: 6.2194 - accuracy: 0.5932 - val_loss: 6.3603 - val_accuracy: 0.5840
Epoch 9/100

INFO:tensorflow:Assets written to: /var/folders/jj/z9jhw1z95m5bwpgdbhgbvct40000gn/T/tmp75l7udqg/model/data/model/assets


🏃 View run bright-ox-333 at: http://127.0.0.1:5000/#/experiments/336063370036033104/runs/061bd9130cf04068aab554dc5f14b61f
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/336063370036033104


In [16]:
model.save(
    "model.h5"
)

/Users/sarthakagarwal/Dropbox/study material/python/genAI/Udemy GenAI/genai/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
